In [52]:
import time
import datetime
import requests
import csv

In [2]:
#!pip install tqdm
from tqdm import notebook

In [3]:
session = requests.Session()
with open ("token.txt", "r") as f:
    x = f.readline()
session.headers.update({"Authorization": "token "+x.strip()})

In [4]:
error_url = []

def FetchDataFromGitHubRestApi(url):
    response = session.get(url)
    if response.status_code == 200:
        fetched_data = response.json()
        poll_interval = None
        poll_interval = response.headers.get('X-Poll-Interval')
        if poll_interval:
            time.sleep(int(poll_interval))
        return fetched_data
    else:
        error_url.append(url)

def Select(field, fetched_data):
    # Select Field From Fetched Data
    if fetched_data[field]:
        if type(fetched_data[field]) == str:
            try:
                output = str(fetched_data[field])
            except:
                output = fetched_data[field]
        else:
            output = str(fetched_data[field])
        output = output.replace("'",'').replace(", ",'/').replace(",",'/').replace('[','').replace(']','')
        if output == " " or output == "":
            output = "None"
    else:
        output = "None"
    return output

In [12]:
with open ("8K_github_username.txt", "r", encoding="utf8") as f:
    all_username = f.readlines()

In [13]:
with open ("not_a_user_or_no_data.txt", "r", encoding="utf8") as f:
    not_a_user_or_no_data = f.readlines()

In [15]:
print(len(all_username), len(not_a_user_or_no_data))

8932 349


In [16]:
for user in not_a_user_or_no_data:
    all_username.remove(user)
print(len(all_username))

8583


In [17]:
all_username = [x.strip() for x in all_username]

In [25]:
user_alias = ["usr"+str(c) for c in range(len(all_username))]

In [26]:
all_event_url = []

for username in all_username:
    for i in (1,2,3):
        all_event_url.append("https://api.github.com/users/"+username+"/events?per_page=100&page="+str(i))

In [27]:
len(all_event_url)

25749

In [29]:
chunk_size = 4999
split_event_url = []
while all_event_url: 
    chunk, all_event_url = all_event_url[:chunk_size], all_event_url[chunk_size:] 
    split_event_url.append(chunk)

In [32]:
len(split_event_url)

6

In [35]:
for x in split_event_url:
    print(len(x))

4999
4999
4999
4999
4999
754


In [36]:
def CollectUserEventData(event_url):
    try:
        event_data = FetchDataFromGitHubRestApi(event_url)
        
        type_ = Select("type", event_data)
        created_at = Select("created_at", event_data)
        return [type_, created_at]
        
    except:
        return ["None", "None"]

In [37]:
print("now ",datetime.datetime.now().hour,":", datetime.datetime.now().minute)

now  13 : 41


In [ ]:
split_number = 1
user_number = 0
user_event_counter = 3
event_dict = {}

for split_event in split_event_url:
    print("Collect User Event Data from Split", split_number)
    split_number += 1
    for event_url in notebook.tqdm(split_event):
        if event_dict[user_alias[user_number]]:
            event_dict[user_alias[user_number]].append(CollectUserEventData(event_url))
        else:
            event_dict[user_alias[user_number]] = []
            event_dict[user_alias[user_number]].append(CollectUserEventData(event_url))
        user_event_counter -= 1
        if user_event_counter == 0:
            user_number += 1
            user_event_counter += 3
    print("Split", split_number, " Complete!")
    print("wait for request limit")
    if split_number < len(split_event_url):
        wait_time = 60 - datetime.datetime.now().minute
        print("wait_time = ", wait_time+1)
        for n in notebook.tqdm(range(wait_time+1)):
            time.sleep(60)
            print("request limit = 4999\n")

print("\ndone")

In [ ]:
print("write to file")

with open('all_user_event_data.csv', 'w', newline='', encoding='utf8') as f:
    
    writer = csv.writer(f)
    
    writer.writerow(["user_alias", "event_type", "created_at"])

    for user in notebook.tqdm(event_dict):
        for user_event in event_dict[user]:
            writer.writerow([user, user_event[0], user_event[1]])

print("done!")

In [ ]:
print("now ",datetime.datetime.now().hour,":", datetime.datetime.now().minute)